https://github.com/BAMresearch/NFDI4IngScientificWorkflowRequirements

In [1]:
from nfdi_ing_workflow import create_conda_environments

In [2]:
env_lst = ["source/envs/preprocessing.yaml", "source/envs/processing.yaml", "source/envs/postprocessing.yaml"]

In [3]:
# create_conda_environments(environment_file_lst=env_lst)

In [4]:
!conda env list

# conda environments:
#
base                  *  /srv/conda/envs/notebook
postprocessing           /srv/conda/envs/notebook/envs/postprocessing
preprocessing            /srv/conda/envs/notebook/envs/preprocessing
processing               /srv/conda/envs/notebook/envs/processing



# Preprocessing 

In [5]:
domain_size = 2.0

## generate mesh

In [6]:
from nfdi_ing_workflow import generate_mesh

In [7]:
gmsh_output_file = generate_mesh(gmsh_input_file="source/unit_square.geo", domain_size=domain_size)
gmsh_output_file

'preprocessing/square.msh'

## convert to xdmf

In [8]:
from nfdi_ing_workflow import convert_to_xdmf

In [9]:
meshio_output = convert_to_xdmf(gmsh_output_file=gmsh_output_file)
meshio_output

'preprocessing/quare.xdmf'

# Processing

## poisson

In [10]:
from nfdi_ing_workflow import poisson

In [11]:
poisson_dict = poisson(meshio_output=meshio_output)
poisson_dict

{'numdofs': 357, 'poisson_output_pvd_file': 'processing/poisson.pvd'}

# Postprocessing

## plot over line

In [12]:
from nfdi_ing_workflow import plot_over_line

In [13]:
pvbatch_output_file = plot_over_line(poisson_output_pvd_file=poisson_dict["poisson_output_pvd_file"])
pvbatch_output_file

'postprocessing/plotoverline.csv'

## substitute macros

In [14]:
from nfdi_ing_workflow import substitute_macros

In [15]:
macros_output_file = substitute_macros(
    macros_source="source/macros.tex.template", 
    pvbatch_output_file=pvbatch_output_file, 
    ndofs=poisson_dict["numdofs"], 
    domain_size=domain_size,
)
macros_output_file

'macros.tex'

## compile paper

In [16]:
from nfdi_ing_workflow import compile_paper

In [17]:
paper_output = compile_paper(paper_source="source/paper.tex")
paper_output

'paper.tex'